# Test `cs_tag.Alignment.accuracy`
This notebook is designed to be run with `nbval` as a test.

Import Python modules:

In [1]:
import collections
import re

import pysam

import alignparse.cs_tag

Create a valid `pysam.AlignedSegment`. 
First, we need a valid header:

In [2]:
header = pysam.AlignmentHeader.from_dict(collections.OrderedDict([
        ('SQ', [{'SN': 'RecA_PacBio_amplicon', 'LN': 1342}]),
        ('PG',
            [{'ID': 'minimap2',
              'PN': 'minimap2',
              'VN': '2.17-r941',
              'CL': ''}])
        ]))

Now create the `pysam.AlignedSegment`:

In [3]:
qual = '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~G~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~G~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~>~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~/~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~R~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

a1_seg = pysam.AlignedSegment.from_dict({
    'name': 'm54228_181120_212724/4194398/ccs',
     'flag': '0',
     'ref_name': 'RecA_PacBio_amplicon',
     'ref_pos': '454',
     'map_quality': '60',
     'cigar': '863S800M1D88M',
     'next_ref_name': '*',
     'next_ref_pos': '0',
     'length': '0',
     'seq': 'GGTACCACACTCTTTCCCTACACGACGCTCTGCCGATCTGAACAATTTGCTTCTTAGTCTAGAATGGCATATCCTTACAACTTAAAAAAAGCAAAAGGGCCGCAGATGCGACCCTTGTGTATCAAACAAGACGATTAAAAATCTTCGTTAGTTTCTGCTACGCCTTCGCTATCATCTACAGAGAAATCCGGCGTTGAGTTCGGGTTGCTCAGCAGCAACTCACGTACTTTCTTCTCGATCTCTTTCGCGGTTTCCGGGTTATCTTTCAGCCAGGCAGTCGCATTCGCTTTACCCTGACCGATCTTCTCACCTTTGTAGCTGTACCACGCGCCTGCTTTCTCGATCAGCTTCTCTTTTACGCCCAGGTCAACCAGTTCGCCGTAGAAGTTGATACCTTCGCCGTAGAGGATCTGGAATTCAGCCTGTTTAAACGGCGCAGCGATTTTGTTCTTCACCACTTTCACGCGGGTTTCGCTACCCACCACGTTTTCGCCCTCTTTCACCGCGCCGATACGACGGATGTCGAGACGAACAGAGGCGTAGAATTTCAGCGCGTTACCACCGGTAGTGGTTTCCGGGTTGCCGAACATCACACCAATTTTCATACGGATCTGGTTGATGAAGATCAGCAGCGTGTTGGACTGCTTCAGGTTACCCGCCAGCTTACGCATCGCCTGGCTCATCATACGTGCCGCAAGGCCCATATGAGAGTCGCCGATTTCGCCTTCGATTTCCGCTTTCGGCGTCAGTGCCGCCACGGAGTCAACGACGATAACGTCTACTGCGCCAGAACGCGCCAGGGCGTCACAGATTTCCAGTGCCTGCTCGCCGGTGTCCGGCTGGGAGCACAGCAGGTTGTCGATATCTACGCACGTAAACTGGGCGTCGATATCGACAACCTGCTGTGCTCCCAGCCGGACACCGGCGAGCAGGCACTGGAAATCTGTGACGCCCTGGCGCGTTCTGGCGCAGTAGACGTTATCGTCGTTGACTCCGTGGCGGCACTGACGCCGAAAGCGGAAATCGAAGGCGAAATCGGCGACTCTCATATGGGCCTTGCGGCACGTATGATGAGCCAGGCGATGCGTAAGCTGGCGGGTAACCTGAAGCAGTCCAACACGCTGCTGATCTTCATCAACCAGATCCGTATGAAAATTGGTGTGATGTTCGGCAACCCGGAAACCACTACCGGTGGTAACGCGCTGAAATTCTACGCCTCTGTTCGTCTCGACATCCGTCGTATCGGCGCGGTGAAAGAGGGCGAAAACGTGGTGGGTAGCGAAACCCGCGTGAAAGTGGTGAAGAACAAAATCGCTGCGCCGTTTAAACAGGCTGAATTCCAGATCCTCTACGGCGAAGGTATCAACTTCTACGGCGAACTGGTTGACCTGGGCGTAAAAGAGAAGCTGATCGAGAAAGCAGGCGCGTGGTACAGCTACAAAGGTGAGAAGATCGGTCAGGGTAAAGCGAATGCGACTGCCTGGCTGAAAGATAACCCGGAAACCGCGAAAGAGATCGAGAAGAAAGTACGTGAGTTGCTGCTGAGCAACCCGAACTCAACGCCGGATTTCTCTGTAGATGATAGCGAAGGCGTAGCAGAAACTAACGAAGATTTTTAATCGTCTTGTTTGATACACAAGGGTCGCATCTGCGGCCCTTTTGCTTTTTAAGTTGTAAGGATATGCCATTCTAGACTAAGAAGCAAATTGTTCAGATCGGCAGAGCGTCGTGTAGGGAAAGAGTGTGGTACC',
     'qual': qual,
     'tags': [
        'NM:i:20',
        'ms:i:1648',
        'AS:i:1648',
        'nn:i:19',
        'tp:A:P',
        'cm:i:167',
        's1:i:841',
        's2:i:0',
        'de:f:0.0011',
        'SA:Z:RecA_PacBio_amplicon,473,-,880S870M1I,60,21;',
        'cs:Z::800-t:31*nc*nt*na*na*ng*na*na*ng*nc*na*na*na*nt*nt*ng*nt*nt*nc:7*nc:31',
        'rl:i:0']},
    header=header)

Convert it to `alignparse.cs_tag.Alignment`:

In [4]:
a1 = alignparse.cs_tag.Alignment(a1_seg)

Accuracy for entire target:

In [5]:
a1.get_accuracy(0, 1342)

0.9999999853063067

No accuracy can be estimated for the first 454 nucleotides of the reference as no query is aligned htere:

In [6]:
a1.get_accuracy(0, 454)

0.9999999994988128

The estimate for the 455th nucleotide is just that one quality score:

In [7]:
a1.get_accuracy(0, 455)

0.9999999994988128

In [8]:
a1.get_accuracy(0, 455) == a1.get_accuracy(455, 456)

True

Look at a lower quality region:

In [9]:
low_q_i, low_q_char = [(m.start(), m.group()) for m in re.finditer('[^~]', qual)][-1]
ref_low_q_i = low_q_i - a1_seg.query_alignment_start + a1_seg.reference_start + 1

In [10]:
low_q_acc = a1.get_accuracy(ref_low_q_i, ref_low_q_i + 1)
low_q_acc

0.999987410745882

In [11]:
low_q_acc == alignparse.utils.qvals_to_accuracy(low_q_char, encoding='sanger')

True

In [12]:
a1.get_accuracy(ref_low_q_i, ref_low_q_i + 2) > low_q_acc

True